<a href="https://colab.research.google.com/github/massilva/dominando-data-science/blob/main/warmup-competicao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd

url = 'https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv'
data = pd.read_csv(url)

data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [53]:
# Transformando os dados
# data['sex'] = data['sex'].map({'female': 0, 'male': 1})
# data['smoker'] = data['smoker'].map({'yes': 1, 'no': 0})
# data['charges'] = data['charges'] / 1000
# data['region'] = data['region'].map({region: i for i, region in enumerate(data['region'].unique())})

# data.head()

In [54]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [55]:
data = pd.get_dummies(data)

In [56]:
data.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0


In [57]:
X = data.drop('charges', axis = 1)
y = data['charges']

In [58]:
X.dtypes

age                   int64
bmi                 float64
children              int64
sex_female            uint8
sex_male              uint8
smoker_no             uint8
smoker_yes            uint8
region_northeast      uint8
region_northwest      uint8
region_southeast      uint8
region_southwest      uint8
dtype: object

In [59]:
y.dtypes

dtype('float64')

In [60]:
def tunagem(modelo, treino, targets, parametros, tipo = 'random', n_iter = 20, scoring = 'accuracy', verbose = 1, n_jobs = -1, validacao = None):
    validacao = validacao or RepeatedKFold(n_splits = 10, n_repeats = 3)
    print(
      'Tunagem:',
      modelo,
      parametros,
      scoring,
      validacao,
      n_iter,
      verbose,
      n_jobs,
    )
    if tipo == 'grid':
      search = GridSearchCV(
        modelo,
        param_grid = parametros,
        scoring = scoring,
        cv = validacao, 
        verbose = verbose,
        n_jobs = n_jobs,
      )
    elif tipo == 'random':
      search = RandomizedSearchCV(
        modelo,
        param_distributions = parametros,
        scoring = scoring,
        cv = validacao,
        n_iter = n_iter,
        verbose = verbose,
        n_jobs = n_jobs,
      )
    else:
      print('entrada invalida no argumento "tipo"')
      return

    search.fit(treino, targets) 
    melhor = search.best_estimator_

    if verbose:
      print('\n{}: {:.4f}'.format(scoring.capitalize(), search.best_score_))
      print('Melhores parâmetros: {}\n'.format(search.best_params_))

    return melhor


In [61]:
parametros = {
  'n_estimators': [50, 100, 150],
  'max_features': range(2, data.columns.size, 2),
  'bootstrap': [True, False] ,
  'max_depth' : [3, 5, 7, 11],
}

modelo = RandomForestRegressor(criterion = 'absolute_error')

rf = tunagem(modelo, X, y, parametros, tipo='random', scoring='neg_mean_absolute_error')
rf

Tunagem: RandomForestRegressor(criterion='absolute_error') {'n_estimators': [50, 100, 150], 'max_features': range(2, 12, 2), 'bootstrap': [True, False], 'max_depth': [3, 5, 7, 11]} neg_mean_absolute_error RepeatedKFold(n_repeats=3, n_splits=10, random_state=None) 20 1 -1
Fitting 30 folds for each of 20 candidates, totalling 600 fits

Neg_mean_absolute_error: -1846.4732
Melhores parâmetros: {'n_estimators': 150, 'max_features': 10, 'max_depth': 5, 'bootstrap': True}



RandomForestRegressor(criterion='absolute_error', max_depth=5, max_features=10,
                      n_estimators=150)

In [62]:
predicts = rf.predict(X)
predicts

array([17556.66063217,  2143.41378483,  4755.984875  , ...,
        1924.54883217,  1962.38122217, 28415.57424283])

In [63]:
mean_absolute_error(y, predicts)

1713.95852181714

In [67]:
#SVM
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [69]:
pipeline = make_pipeline(
  StandardScaler(),
  LinearSVR(random_state=0, tol=1e-5),
)
pipeline.fit(X, y)
predicts = pipeline.predict(X)
predicts

array([1354.56343762, 1170.74918436, 1219.06361685, ..., 1227.45874031,
       1313.74878655, 1447.69426392])

In [70]:
mean_absolute_error(y, predicts)

11951.16962589473